In [18]:
import pandas as pd
import numpy as np
import json

In [19]:
class Node():
    def  __init__(self, 
                  s        = None, 
                  leftchild  = None,
                  rightchild = None,
                  f = None):
        self.s        = s
        self.leftchild  = leftchild
        self.rightchild = rightchild
        self.f = f

def dump(root: Node) -> str:
    def _to_dict(node) -> dict:    
        return {
            "stock price": node.s,
            "option price": node.f,
            "down": (_to_dict(node.leftchild) if node.leftchild is not None else None),
            "up": (_to_dict(node.rightchild) if node.rightchild is not None else None)
        }
    if root == None:
        dict_repr = {}
    else:
        dict_repr = _to_dict(root)
    return json.dumps(dict_repr)

In [20]:
t = Node(s=50, f=45)
t.leftchild = Node(s=40, f=40)
t.rightchild = Node(s=60, f=50)
dump(t)

'{"stock price": 50, "option price": 45, "down": {"stock price": 40, "option price": 40, "down": null, "up": null}, "up": {"stock price": 60, "option price": 50, "down": null, "up": null}}'